Con este programa se busca tener una lista de las Rutas y los baños que se tienen en la ciudad, además de ir viendo cómo podemos escalar el programa de rutas

**PUNTOS IMPORTANTES**

- Que este código lo puedan correr sin que yo no esté, pero teniendo ellos el código
- Que no se pueda modificar ni checar para modificar el código fuente
- Poder visualizar como tenemos la ruta y como la arroja el código


In [43]:
import requests
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
import polyline
from datetime import timedelta
from math import radians, sin, cos, sqrt, atan2

# Coordenadas de las ubicaciones (latitud, longitud)
ubicaciones = [
    (24.772824, -107.443294),  # drenax 0 
    (24.823004, -107.383656),  # Botanico 1
    (24.794996, -107.433254),  # Panama pedro infante 2
    (24.827219, -107.426344),  # Sendero 3
    (24.778562, -107.394249),  # Walmart 4
]

# Número de baños portátiles en cada ubicación
num_banos = [1, 3, 3, 2, 4]  # Ejemplo: 1 baño en la primera ubicación, 3 en la segunda, etc.

# Nombres de los clientes en cada ubicación
clientes = [
    "drenax",               # drenax 0 
    "Botanico",             # Botanico 1
    "Panama pedro infante", # Panama pedro infante 2
    "Sendero",              # Sendero 3
    "Walmart",              # Walmart 4
]

# Función para calcular la distancia haversine entre dos puntos (en kilómetros)
def haversine(lat1, lon1, lat2, lon2):
    R = 6371.0  # Radio de la Tierra en kilómetros
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distancia = R * c
    return distancia

# Función para obtener la matriz de tiempos usando OSRM
def obtener_matriz_tiempos(ubicaciones):
    n = len(ubicaciones)
    matriz_tiempos = np.zeros((n, n))

    # Construir la URL para la solicitud a OSRM
    coordenadas = ";".join([f"{lon},{lat}" for lat, lon in ubicaciones])
    url = f"http://router.project-osrm.org/table/v1/driving/{coordenadas}"

    # Parámetros de la solicitud
    params = {
        "sources": ";".join(map(str, range(n))),  # Todas las ubicaciones como fuentes
        "destinations": ";".join(map(str, range(n))),  # Todas las ubicaciones como destinos
    }

    # Hacer la solicitud a OSRM
    response = requests.get(url, params=params)
    if response.status_code == 200:
        datos = response.json()
        matriz_tiempos = np.array(datos["durations"])  # Matriz de tiempos en segundos
    else:
        raise Exception(f"Error al obtener la matriz de tiempos: {response.status_code}")

    return matriz_tiempos

# Función para resolver la ruta más óptima (Problema del Viajante - TSP)
def resolver_ruta(matriz_distancias):
    n = len(matriz_distancias)
    manager = pywrapcp.RoutingIndexManager(n, 1, 0)  # 1 vehículo, punto de inicio 0
    routing = pywrapcp.RoutingModel(manager)

    def distancia_callback(from_index, to_index):
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return matriz_distancias[from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distancia_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Configuración del solver
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC
    )

    # Resolver el problema
    solution = routing.SolveWithParameters(search_parameters)

    # Extraer la ruta optimizada
    ruta_optimizada = []
    index = routing.Start(0)
    while not routing.IsEnd(index):
        ruta_optimizada.append(manager.IndexToNode(index))
        index = solution.Value(routing.NextVar(index))
    ruta_optimizada.append(manager.IndexToNode(index))

    return ruta_optimizada

# Función para obtener las coordenadas de la ruta entre dos puntos usando OSRM
def obtener_ruta_entre_puntos(origen, destino):
    url = f"http://router.project-osrm.org/route/v1/driving/{origen[1]},{origen[0]};{destino[1]},{destino[0]}"
    response = requests.get(url)
    if response.status_code == 200:
        datos = response.json()
        return datos["routes"][0]["geometry"]  # Geometría de la ruta en formato polyline
    else:
        raise Exception(f"Error al obtener la ruta: {response.status_code}")

# Obtener la matriz de tiempos
matriz_tiempos = obtener_matriz_tiempos(ubicaciones)

# Calcular la matriz de distancias usando la fórmula haversine
n = len(ubicaciones)
matriz_distancias = np.zeros((n, n))
for i in range(n):
    for j in range(n):
        if i != j:
            lat1, lon1 = ubicaciones[i]
            lat2, lon2 = ubicaciones[j]
            matriz_distancias[i][j] = haversine(lat1, lon1, lat2, lon2)  # Distancia en kilómetros

# Resolver la ruta optimizada
ruta_optimizada = resolver_ruta(matriz_distancias)

# Mostrar la ruta optimizada con nombres de ubicación, latitud y longitud
print("Ruta optimizada:")
for i, punto in enumerate(ruta_optimizada):
    nombre_cliente = clientes[punto]
    latitud, longitud = ubicaciones[punto]
    print(f"Parada {i+1}: {nombre_cliente} - ({latitud}, {longitud})")

# Calcular la distancia total recorrida en kilómetros
distancia_total = 0
for i in range(len(ruta_optimizada) - 1):
    origen = ruta_optimizada[i]
    destino = ruta_optimizada[i + 1]
    distancia_total += matriz_distancias[origen][destino]

print(f"Distancia total recorrida: {distancia_total:.2f} km")

# Calcular el tiempo total estimado del recorrido
tiempo_total = 0
for i in range(len(ruta_optimizada) - 1):
    origen = ruta_optimizada[i]
    destino = ruta_optimizada[i + 1]
    tiempo_total += matriz_tiempos[origen][destino]  # Sumar tiempos en segundos

# Convertir el tiempo total a un formato legible (horas, minutos, segundos)
tiempo_total_formateado = str(timedelta(seconds=tiempo_total))
print(f"Tiempo estimado del recorrido: {tiempo_total_formateado}")

# Crear un mapa con Folium
mapa = folium.Map(location=ubicaciones[0], zoom_start=13)

# Dibujar la ruta optimizada en el mapa
for i in range(len(ruta_optimizada) - 1):
    origen = ubicaciones[ruta_optimizada[i]]
    destino = ubicaciones[ruta_optimizada[i + 1]]
    ruta_geometria = obtener_ruta_entre_puntos(origen, destino)
    # Decodificar la geometría de la ruta usando polyline
    ruta_coordenadas = polyline.decode(ruta_geometria)
    folium.PolyLine(
        locations=ruta_coordenadas,
        color="blue",
        weight=2.5,
        opacity=1,
    ).add_to(mapa)

# Añadir marcadores para cada parada
for i, punto in enumerate(ruta_optimizada):
    # Crear el contenido del popup con HTML y CSS
    popup_content = f"""
    <div style="font-size: 16px; width: 200px; height: 80px;">
        <strong>Cliente:</strong> {clientes[punto]}<br>
        <strong>Baños:</strong> {num_banos[punto]}
    </div>
    """
    
    folium.Marker(
        location=ubicaciones[punto],
        popup=folium.Popup(popup_content, max_width=250),  # Ajustar el ancho máximo del popup
        tooltip=f"Parada {i+1}",  # Tooltip con el número de parada
        icon=folium.Icon(color="green" if i == 0 else "red", icon="info-sign"),
    ).add_to(mapa)

# Mostrar el mapa directamente en la salida del script
mapa

Ruta optimizada:
Parada 1: drenax - (24.772824, -107.443294)
Parada 2: Panama pedro infante - (24.794996, -107.433254)
Parada 3: Sendero - (24.827219, -107.426344)
Parada 4: Botanico - (24.823004, -107.383656)
Parada 5: Walmart - (24.778562, -107.394249)
Parada 6: drenax - (24.772824, -107.443294)
Distancia total recorrida: 20.70 km
Tiempo estimado del recorrido: 0:54:08


In [50]:
import gspread
from google.oauth2.service_account import Credentials
import requests
import numpy as np
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import folium
import polyline
from datetime import timedelta
from math import radians, sin, cos, sqrt, atan2
import streamlit as st

# Configurar las credenciales y el alcance
SCOPES = [
    "https://www.googleapis.com/auth/spreadsheets",
    "https://www.googleapis.com/auth/drive",
]

# Ruta al archivo JSON de credenciales
CREDENTIALS_FILE = "credenciales.json"  # Cambia esto por la ruta correcta

# Autenticar con Google Sheets
creds = Credentials.from_service_account_file(CREDENTIALS_FILE, scopes=SCOPES)
client = gspread.authorize(creds)

# Abrir la hoja de cálculo por su nombre o URL
spreadsheet = client.open("Nombre de tu hoja de cálculo")  # Cambia esto por el nombre de tu hoja

# Seleccionar la hoja de trabajo (pestaña) por su nombre
worksheet = spreadsheet.worksheet("Hoja1")  # Cambia "Hoja1" por el nombre de tu pestaña

# Leer todos los datos de la hoja
datos = worksheet.get_all_records()

# Extraer las columnas necesarias
ubicaciones = [(float(fila["Latitud"]), float(fila["Longitud"])) for fila in datos]
clientes = [fila["Cliente"] for fila in datos]
num_banos = [int(fila["Baños"]) for fila in datos]

# Resto del código de optimización de rutas (usando ubicaciones, clientes y num_banos)
# ... (Inserta aquí el código de optimización de rutas que ya tienes)

# Mostrar resultados en Streamlit
st.title("Optimización de Rutas para Limpieza de Baños Portátiles")

# Mostrar la ruta optimizada
st.subheader("Ruta Optimizada")
for i, punto in enumerate(ruta_optimizada):
    nombre_cliente = clientes[punto]
    latitud, longitud = ubicaciones[punto]
    st.write(f"Parada {i+1}: {nombre_cliente} - ({latitud}, {longitud})")

# Mostrar la distancia total y el tiempo estimado
st.subheader("Resumen del Recorrido")
st.write(f"Distancia total recorrida: {distancia_total:.2f} km")
st.write(f"Tiempo estimado del recorrido: {tiempo_total_formateado}")

# Mostrar el mapa en Streamlit
st.subheader("Mapa de la Ruta Optimizada")
st.write(mapa._repr_html_(), unsafe_allow_html=True)

FileNotFoundError: [Errno 2] No such file or directory: 'credenciales.json'

In [45]:
pip install gspread

  Attempting uninstall: google-auth
    Found existing installation: google-auth 1.33.0
    Uninstalling google-auth-1.33.0:
      Successfully uninstalled google-auth-1.33.0
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-storage 1.31.0 requires google-auth<2.0dev,>=1.11.0, but you have google-auth 2.38.0 which is incompatible.
google-cloud-core 1.7.1 requires google-auth<2.0dev,>=1.24.0, but you have google-auth 2.38.0 which is incompatible.
google-api-core 1.25.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 2.38.0 which is incompatible.


In [46]:
pip install streamlit


Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: jinja2 in c:\users\erick\anaconda\lib\site-packages (from altair<6,>=4.0->streamlit) (3.1.3)


ERROR: Could not install packages due to an OSError: [WinError 5] Acceso denegado: 'C:\\Users\\erick\\Anaconda\\Lib\\site-packages\\~-mpy\\.libs\\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll'
Consider using the `--user` option or check the permissions.




  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.11.2
    Uninstalling Pygments-2.11.2:
      Successfully uninstalled Pygments-2.11.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
